# More Image Recognition: Cat or Dog?

The dataset can be downloaded from [Microsoft](https://www.microsoft.com/en-us/download/details.aspx?id=54765).

## Preamble

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import cv2

## Preparing the Data

In [ ]:
import os
import random
import pandas

In [ ]:
data_dir = "/Users/cls/Downloads/kagglecatsanddogs_3367a/PetImages/"

In [ ]:
categories = ["Dog", "Cat"]
labels = dict(zip(categories, range(len(categories))))

In [ ]:
def load_images(data_dir, categories, n_imgs):
    """Expects data_dir to be a directory with one subfolder for each category"""
    imgs = []
    for category in categories:
        path = os.path.join(data_dir, category)
        count = 0
        for img_file in os.listdir(path):
            img_path = os.path.join(path, img_file)
            try:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                row = {"img": img, "category": category, "path": img_path}
                imgs.append(row)
                count += 1
            except Exception as ex:
                # some of the image files are broken - just ignore them
                print(ex, f": {img_path}")
            if count >= (n_imgs / len(categories)):
                break
    img_df = pandas.DataFrame(imgs, columns=["img", "category", "path"])
    category_labels = dict(zip(categories, range(len(categories))))
    img_df["label"] = img_df["category"].apply(lambda c: category_labels[c])
    img_df = img_df.dropna()
    return img_df


In [ ]:
%time
imgs = load_images(data_dir=data_dir, categories=categories, n_imgs=4000)

In [ ]:
imgs.head()

In [ ]:
def show_images(imgs, img_col, label_col, n=10):
    random_imgs = imgs.sample(n)
    for (i, row) in random_imgs.iterrows():
        plt.imshow(row[img_col], cmap="gray")
        plt.title(row[label_col])
        plt.show()

In [ ]:
show_images(imgs, img_col="img", label_col="category", n=10)

## Resizing

In [ ]:
img_size = 80

In [ ]:
def resize_image(img, img_size):
    try:
        res_img = cv2.resize(img, (img_size, img_size))
        return res_img
    except:
        return None

In [ ]:
imgs["img_res"] = imgs["img"].apply(lambda img: resize_image(img, img_size))

In [ ]:
show_images(imgs, img_col="img_res", label_col="category", n=10)

## Building the Training Set

In [ ]:
import numpy as np

In [ ]:
def tensorize(imgs):
    """Prepare as tensors X, y to feed to the network"""
    imgs = imgs.sample(frac=1) # shuffle
    X = np.array([img for img in imgs["img_res"]])
    X = X.reshape(-1, img_size, img_size, 1)
    X = X / 255 # normalize
    y = np.array([label for label in imgs["label"]])
    return X, y

In [ ]:
X, y = tensorize(imgs)

## Building the Network

In [ ]:
X.shape[1:]

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=64, 
                           kernel_size=(3,3), 
                           input_shape=X.shape[1:],
                           activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=64, 
                           kernel_size=(3,3), 
                           activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, activation="relu"),
    tf.keras.layers.Dense(units=1, activation="sigmoid")
])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
%%time
model.fit(X, y,
          batch_size=32,
          validation_split=0.2,
          epochs=1, 
          callbacks=[
              tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.01),
              tf.keras.callbacks.TensorBoard(log_dir='./Graph', 
                                             histogram_freq=0, 
                                             write_graph=True, 
                                             write_images=True)
          ],
        )

In [ ]:
X.shape

In [ ]:
y.shape

## Test Set

In [ ]:
%time
test_imgs = load_images(data_dir="/Users/cls/Desktop/catdog/", categories=categories, n_imgs=500)

In [ ]:
test_imgs.head()

In [ ]:
show_images(test_imgs, img_col="img", label_col="category", n=10)

In [ ]:
test_imgs["img_res"] = test_imgs["img"].apply(lambda img: resize_image(img, img_size))

In [ ]:
X_test, y_test = tensorize(test_imgs)

In [ ]:
y_pred = model.predict_classes(X_test)

In [ ]:
inv_labels = {v: k for k, v in labels.items()}

In [ ]:
test_imgs["label_pred"] = pandas.Series(y_pred.flatten(), index=test_imgs.index)
test_imgs["category_pred"] = test_imgs["label_pred"].apply(lambda l: inv_labels[l])

In [ ]:
test_imgs.head(20)

In [ ]:
show_images(test_imgs, img_col="img", label_col="category_pred", n=20)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_true=test_imgs["label"], y_pred=test_imgs["label_pred"])
precision = precision_score(y_true=test_imgs["label"], y_pred=test_imgs["label_pred"])
recall = recall_score(y_true=test_imgs["label"], y_pred=test_imgs["label_pred"])
f1 = f1_score(y_true=test_imgs["label"], y_pred=test_imgs["label_pred"])

In [ ]:
print(f"""
accuracy: {accuracy}
precision: {precision}
recall : {recall}
f1 : {f1}
""")

---
_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © 2018-2022 [Point 8 GmbH](https://point-8.de)_